In [1]:
# Delete模型自动化持续运行系统 - 基于运行时长版本
# 功能：设置运行时长，自动重启，避免输出覆盖，等待最后一次运行完成

import subprocess
import os
import time
import datetime
from pathlib import Path
import shutil

# ==============================================================================
# 配置区：请修改以下参数
# ==============================================================================

# 【必填】Delete模型根目录的绝对路径
DELETE_ROOT = r"/home/cenking/VsCode/Delete/Delete_Running"

# 【必填】Delete运行参数配置
DELETE_CONFIG = {
    'surf_path': './data/TDFA/TDFA_Macro_pocket_8.0.ply',
    'frag_path': './data/TDFA/TDFA1_Coor.sdf',
    'check_point': './checkpoint/ckpt/delete.pt',
    'outdir': './outputs',
    'suboutdir': 'TDFA1'  # 基础输出文件夹名称
}

# 【必填】运行时长设置（小时）
RUN_DURATION_HOURS = 1  # 运行7小时后停止（可改为 8, 10 等）

# 【可选】是否使用CPU版本（True/False）
USE_CPU_VERSION = False  # 如果为True，将使用delete_cpu.py

# 【可选】每次运行之间的等待时间（秒）
WAIT_BETWEEN_RUNS = 10

# 【可选】输出文件夹命名模式
# 可用变量: {base_name}, {run_number}, {timestamp}, {date}, {time}
OUTPUT_NAMING_PATTERN = "{base_name}_Run_{run_number}_{timestamp}"

# ==============================================================================
# 核心功能实现
# ==============================================================================

class DeleteAutoRunner:
    """Delete模型自动化运行管理器 - 基于运行时长版本"""
    
    def __init__(self, delete_root, config, duration_hours, use_cpu=False):
        self.delete_root = Path(delete_root)
        self.config = config.copy()
        self.duration_hours = duration_hours
        self.use_cpu = use_cpu
        self.run_counter = 1
        self.total_runs = 0
        
        # 验证Delete根目录
        if not self.delete_root.exists():
            raise ValueError(f"Delete根目录不存在: {self.delete_root}")
        
        # 计算开始和结束时间
        self.start_time = datetime.datetime.now()
        self.end_time = self.start_time + datetime.timedelta(hours=duration_hours)
        
        print("=" * 80)
        print("🚀 Delete模型自动化持续运行系统已启动（基于运行时长）")
        print("=" * 80)
        print(f"📂 Delete根目录: {self.delete_root}")
        print(f"⏰ 开始时间: {self.start_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"⏰ 预计结束时间: {self.end_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"⏱️  运行时长: {duration_hours} 小时")
        print(f"💻 运行模式: {'CPU版本' if use_cpu else 'GPU版本'}")
        print(f"🎯 基础输出目录: {config['suboutdir']}")
        print("=" * 80)
    
    def should_start_new_run(self):
        """
        检查是否应该开始新的运行
        注意：这里只检查开始时是否超时，已经开始的运行会继续完成
        """
        current_time = datetime.datetime.now()
        return current_time < self.end_time
    
    def get_remaining_time(self):
        """获取剩余时间（秒）"""
        current_time = datetime.datetime.now()
        remaining = (self.end_time - current_time).total_seconds()
        return max(0, remaining)
    
    def format_time(self, seconds):
        """格式化时间显示"""
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        return f"{hours}小时{minutes}分{secs}秒"
    
    def generate_output_folder_name(self):
        """生成唯一的输出文件夹名称"""
        base_name = self.config['suboutdir']
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        date = datetime.datetime.now().strftime("%Y%m%d")
        time_str = datetime.datetime.now().strftime("%H%M%S")
        
        # 使用配置的命名模式
        folder_name = OUTPUT_NAMING_PATTERN.format(
            base_name=base_name,
            run_number=self.run_counter,
            timestamp=timestamp,
            date=date,
            time=time_str
        )
        
        return folder_name
    
    def build_command(self, output_folder):
        """构建Delete运行命令"""
        script_name = "delete_cpu.py" if self.use_cpu else "delete.py"
        
        cmd = [
            "python", "-u", script_name,
            "--surf_path", self.config['surf_path'],
            "--frag_path", self.config['frag_path'],
            "--check_point", self.config['check_point'],
            "--outdir", self.config['outdir'],
            "--suboutdir", output_folder
        ]
        
        return cmd
    
    def run_delete_once(self):
        """执行一次Delete模型运行"""
        # 生成唯一的输出文件夹名称
        output_folder = self.generate_output_folder_name()
        
        print(f"\n{'='*80}")
        print(f"🔄 开始第 {self.run_counter} 次运行")
        print(f"📁 输出文件夹: {output_folder}")
        print(f"🕐 开始时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"⏳ 剩余时间: {self.format_time(self.get_remaining_time())}")
        print(f"{'='*80}\n")
        
        # 构建命令
        cmd = self.build_command(output_folder)
        
        try:
            # 切换到Delete根目录
            os.chdir(self.delete_root)
            
            # 执行命令
            print(f"执行命令: {' '.join(cmd)}\n")
            
            start_time = time.time()
            process = subprocess.Popen(
                cmd,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True,
                bufsize=1
            )
            
            # 实时打印输出
            for line in process.stdout:
                print(line, end='')
            
            # 等待进程结束（即使超时也要等待完成）
            return_code = process.wait()
            end_time = time.time()
            duration = end_time - start_time
            
            print(f"\n{'='*80}")
            print(f"✅ 第 {self.run_counter} 次运行完成")
            print(f"⏱️  运行时长: {duration/60:.2f} 分钟")
            print(f"📊 返回代码: {return_code}")
            print(f"🕐 结束时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            
            # 检查输出文件夹是否存在
            output_path = self.delete_root / self.config['outdir'] / output_folder
            if output_path.exists():
                # 统计生成的分子数量
                sdf_files = list(output_path.glob("*.sdf"))
                print(f"📦 生成文件数: {len(sdf_files)}")
            else:
                print(f"⚠️  警告: 输出文件夹未找到: {output_path}")
            
            # 显示剩余时间
            remaining = self.get_remaining_time()
            print(f"⏳ 剩余总时间: {self.format_time(remaining)}")
            print(f"{'='*80}\n")
            
            self.run_counter += 1
            self.total_runs += 1
            
            return True
            
        except Exception as e:
            print(f"\n{'='*80}")
            print(f"❌ 第 {self.run_counter} 次运行出错")
            print(f"错误信息: {e}")
            print(f"{'='*80}\n")
            return False
    
    def run_continuous(self):
        """持续运行Delete模型"""
        print("\n🎬 开始持续运行循环...\n")
        
        try:
            while True:
                # 检查是否应该开始新的运行
                if not self.should_start_new_run():
                    print(f"⏰ 已达到设定的运行时长 ({self.duration_hours}小时)")
                    print(f"🛑 不再开始新的运行，准备结束...")
                    break
                
                # 执行一次运行（即使运行过程中超时，也会完成这次运行）
                print(f"✅ 开始新的运行（剩余时间: {self.format_time(self.get_remaining_time())}）")
                success = self.run_delete_once()
                
                # 再次检查时间（运行完成后）
                if not self.should_start_new_run():
                    print(f"\n⏰ 运行完成后已超过设定时长")
                    print(f"🛑 不再开始新的运行")
                    break
                
                # 等待一段时间后再次运行
                if WAIT_BETWEEN_RUNS > 0:
                    print(f"⏳ 等待 {WAIT_BETWEEN_RUNS} 秒后开始下一次运行...\n")
                    time.sleep(WAIT_BETWEEN_RUNS)
                
        except KeyboardInterrupt:
            print(f"\n{'='*80}")
            print("🛑 用户中断运行")
            
        finally:
            # 计算实际运行时间
            actual_duration = (datetime.datetime.now() - self.start_time).total_seconds() / 3600
            
            print(f"\n{'='*80}")
            print("🏁 运行结束")
            print(f"📊 总共完成运行次数: {self.total_runs}")
            print(f"⏱️  设定运行时长: {self.duration_hours} 小时")
            print(f"⏱️  实际运行时长: {actual_duration:.2f} 小时")
            print(f"🕐 开始时间: {self.start_time.strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"🕐 结束时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"{'='*80}")

# ==============================================================================
# 执行自动化运行
# ==============================================================================

if __name__ == "__main__":
    # 创建自动运行管理器
    runner = DeleteAutoRunner(
        delete_root=DELETE_ROOT,
        config=DELETE_CONFIG,
        duration_hours=RUN_DURATION_HOURS,
        use_cpu=USE_CPU_VERSION
    )
    
    # 开始持续运行
    runner.run_continuous()

# ==============================================================================
# 使用说明
# ==============================================================================
"""
📖 使用指南（基于运行时长版本）：

1. 【配置参数】
   - 修改DELETE_ROOT为您的Delete模型根目录
   - 修改DELETE_CONFIG中的路径参数
   - 设置RUN_DURATION_HOURS定义运行时长（单位：小时）
   
2. 【运行时长设置】
   - RUN_DURATION_HOURS = 7: 运行7小时
   - RUN_DURATION_HOURS = 8: 运行8小时
   - RUN_DURATION_HOURS = 10: 运行10小时
   - 可以使用小数，例如 RUN_DURATION_HOURS = 0.5 表示30分钟
   
3. 【运行逻辑】
   - 系统会在设定时长内持续运行Delete模型
   - 如果最后一次运行开始时还没超时，会等待其完成
   - 这意味着实际运行时间可能会略超过设定时长
   
4. 【运行模式】
   - USE_CPU_VERSION = True: 使用delete_cpu.py（无GPU）
   - USE_CPU_VERSION = False: 使用delete.py（有GPU）
   
5. 【输出文件夹命名】
   - 默认格式: 7wmm_Frag2_Run_1_20251112_223045
   - 可修改OUTPUT_NAMING_PATTERN自定义命名
   
6. 【停止运行】
   - 自动停止: 达到设定时长后自动结束（等待当前运行完成）
   - 手动停止: 在Jupyter中点击停止按钮
   
7. 【监控运行】
   - 实时显示Delete模型输出
   - 显示每次运行的时长和文件数
   - 显示剩余运行时间
   - 统计总运行次数和实际运行时长

⚠️  注意事项：
   - 确保Delete环境已激活
   - 确保所有输入文件路径正确
   - 建议先手动运行一次测试
   - 磁盘空间充足（每次运行可能生成大量文件）
   - 实际运行时间 = 设定时长 + 最后一次运行的时间
   
💡 示例：
   如果设置 RUN_DURATION_HOURS = 7，每次运行需要30分钟
   - 第1次运行: 0:00 - 0:30 ✓
   - 第2次运行: 0:30 - 1:00 ✓
   - ...
   - 第14次运行: 6:30 - 7:00 ✓
   - 第15次运行: 7:00 开始时未超时，会继续运行到 7:30 完成 ✓
   - 第16次运行: 7:30 超时，不再开始 ✗
   实际总运行时间: 约7.5小时
"""

🚀 Delete模型自动化持续运行系统已启动（基于运行时长）
📂 Delete根目录: /home/cenking/VsCode/Delete/Delete_Running
⏰ 开始时间: 2025-12-07 18:26:45
⏰ 预计结束时间: 2025-12-07 19:26:45
⏱️  运行时长: 1 小时
💻 运行模式: GPU版本
🎯 基础输出目录: TDFA1

🎬 开始持续运行循环...

✅ 开始新的运行（剩余时间: 0小时59分59秒）

🔄 开始第 1 次运行
📁 输出文件夹: TDFA1_Run_1_20251207_182645
🕐 开始时间: 2025-12-07 18:26:45
⏳ 剩余时间: 0小时59分59秒

执行命令: python -u delete.py --surf_path ./data/TDFA/TDFA_Macro_pocket_8.0.ply --frag_path ./data/TDFA/TDFA1_Coor.sdf --check_point ./checkpoint/ckpt/delete.pt --outdir ./outputs --suboutdir TDFA1_Run_1_20251207_182645

Num of parameters is 37.06M
Start to generate!
/home/cenking/anaconda3/envs/Delete/lib/python3.9/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4322.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Success: NC(=O)n1c2ccccc2c2ccccc21
Success: FN=Cn1c2ccccc2c2ccccc

'\n📖 使用指南（基于运行时长版本）：\n\n1. 【配置参数】\n   - 修改DELETE_ROOT为您的Delete模型根目录\n   - 修改DELETE_CONFIG中的路径参数\n   - 设置RUN_DURATION_HOURS定义运行时长（单位：小时）\n   \n2. 【运行时长设置】\n   - RUN_DURATION_HOURS = 7: 运行7小时\n   - RUN_DURATION_HOURS = 8: 运行8小时\n   - RUN_DURATION_HOURS = 10: 运行10小时\n   - 可以使用小数，例如 RUN_DURATION_HOURS = 0.5 表示30分钟\n   \n3. 【运行逻辑】\n   - 系统会在设定时长内持续运行Delete模型\n   - 如果最后一次运行开始时还没超时，会等待其完成\n   - 这意味着实际运行时间可能会略超过设定时长\n   \n4. 【运行模式】\n   - USE_CPU_VERSION = True: 使用delete_cpu.py（无GPU）\n   - USE_CPU_VERSION = False: 使用delete.py（有GPU）\n   \n5. 【输出文件夹命名】\n   - 默认格式: 7wmm_Frag2_Run_1_20251112_223045\n   - 可修改OUTPUT_NAMING_PATTERN自定义命名\n   \n6. 【停止运行】\n   - 自动停止: 达到设定时长后自动结束（等待当前运行完成）\n   - 手动停止: 在Jupyter中点击停止按钮\n   \n7. 【监控运行】\n   - 实时显示Delete模型输出\n   - 显示每次运行的时长和文件数\n   - 显示剩余运行时间\n   - 统计总运行次数和实际运行时长\n\n⚠️  注意事项：\n   - 确保Delete环境已激活\n   - 确保所有输入文件路径正确\n   - 建议先手动运行一次测试\n   - 磁盘空间充足（每次运行可能生成大量文件）\n   - 实际运行时间 = 设定时长 + 最后一次运行的时间\n   \n💡 示例：\n   如果设置 RUN_DURATION_HOURS = 7，每次运行需要30分钟\n   - 第1次运行: 0:0

In [ ]:
# Delete模型自动化持续运行系统 - 基于运行时长版本
# 功能：设置运行时长，静默运行，输出log文件，tqdm进度条显示

import subprocess
import os
import time
import datetime
from pathlib import Path
import logging
from tqdm.notebook import tqdm

# ==============================================================================
# 配置区：请修改以下参数
# ==============================================================================

# 【必填】Delete模型根目录的绝对路径
DELETE_ROOT = r"/home/cenking/VsCode/Delete"

# 【必填】Delete运行参数配置
DELETE_CONFIG = {
    'surf_path': './data/workdir/7wmm/7WMM_Macro_pocket_8.0.ply',
    'frag_path': './data/workdir/7wmm/Frag2.sdf',
    'check_point': './checkpoint/ckpt/delete.pt',
    'outdir': './outputs',
    'suboutdir': '7wmm_Frag2'  # 基础输出文件夹名称
}

# 【必填】运行时长设置（小时）
RUN_DURATION_HOURS = 6  # 运行6小时后自动停止

# 【可选】是否使用CPU版本（True/False）
USE_CPU_VERSION = True  # 如果为True，将使用delete_cpu.py

# 【可选】每次运行之间的等待时间（秒）
WAIT_BETWEEN_RUNS = 10

# 【可选】输出文件夹命名模式
OUTPUT_NAMING_PATTERN = "{base_name}_Run_{run_number}_{timestamp}"

# 【可选】日志文件路径
LOG_FILE = "Delete_AutoRun.log"

# ==============================================================================
# 核心功能实现
# ==============================================================================

class DeleteAutoRunnerV2:
    """Delete模型自动化运行管理器 - 基于运行时长版本"""
    
    def __init__(self, delete_root, config, duration_hours, use_cpu=False, log_file=None):
        self.delete_root = Path(delete_root)
        self.config = config.copy()
        self.duration_hours = duration_hours
        self.use_cpu = use_cpu
        self.run_counter = 1
        self.total_runs = 0
        self.log_file = log_file
        
        # 验证Delete根目录
        if not self.delete_root.exists():
            raise ValueError(f"Delete根目录不存在: {self.delete_root}")
        
        # 设置日志
        self._setup_logging()
        
        # 计算结束时间
        self.start_time = datetime.datetime.now()
        self.end_time = self.start_time + datetime.timedelta(hours=duration_hours)
        
        # 记录启动信息
        self.log("=" * 80)
        self.log("🚀 Delete模型自动化持续运行系统已启动（基于运行时长）")
        self.log("=" * 80)
        self.log(f"📂 Delete根目录: {self.delete_root}")
        self.log(f"⏰ 开始时间: {self.start_time.strftime('%Y-%m-%d %H:%M:%S')}")
        self.log(f"⏰ 预计结束时间: {self.end_time.strftime('%Y-%m-%d %H:%M:%S')}")
        self.log(f"⏱️  运行时长: {duration_hours} 小时")
        self.log(f"💻 运行模式: {'CPU版本' if use_cpu else 'GPU版本'}")
        self.log(f"🎯 基础输出目录: {config['suboutdir']}")
        self.log(f"📝 日志文件: {self.log_file}")
        self.log("=" * 80)
        
        print(f"✅ 系统已启动，将运行 {duration_hours} 小时")
        print(f"📝 详细日志保存在: {self.log_file}")
        print(f"⏰ 预计结束时间: {self.end_time.strftime('%Y-%m-%d %H:%M:%S')}")
    
    def _setup_logging(self):
        """设置日志系统"""
        if self.log_file:
            log_path = self.delete_root / self.log_file
        else:
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            log_path = self.delete_root / f"Delete_AutoRun_{timestamp}.log"
        
        self.log_file = str(log_path)
        
        # 配置日志格式
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(self.log_file, encoding='utf-8'),
            ]
        )
        self.logger = logging.getLogger(__name__)
    
    def log(self, message):
        """写入日志"""
        self.logger.info(message)
    
    def is_time_remaining(self):
        """检查是否还有剩余运行时间"""
        current_time = datetime.datetime.now()
        return current_time < self.end_time
    
    def get_remaining_time(self):
        """获取剩余时间（秒）"""
        current_time = datetime.datetime.now()
        remaining = (self.end_time - current_time).total_seconds()
        return max(0, remaining)
    
    def format_time(self, seconds):
        """格式化时间显示"""
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        return f"{hours}小时{minutes}分{secs}秒"
    
    def generate_output_folder_name(self):
        """生成唯一的输出文件夹名称"""
        base_name = self.config['suboutdir']
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        date = datetime.datetime.now().strftime("%Y%m%d")
        time_str = datetime.datetime.now().strftime("%H%M%S")
        
        folder_name = OUTPUT_NAMING_PATTERN.format(
            base_name=base_name,
            run_number=self.run_counter,
            timestamp=timestamp,
            date=date,
            time=time_str
        )
        
        return folder_name
    
    def build_command(self, output_folder):
        """构建Delete运行命令"""
        script_name = "delete_cpu.py" if self.use_cpu else "delete.py"
        
        cmd = [
            "python", "-u", script_name,
            "--surf_path", self.config['surf_path'],
            "--frag_path", self.config['frag_path'],
            "--check_point", self.config['check_point'],
            "--outdir", self.config['outdir'],
            "--suboutdir", output_folder
        ]
        
        return cmd
    
    def run_delete_once(self):
        """执行一次Delete模型运行（静默模式）"""
        output_folder = self.generate_output_folder_name()
        
        self.log("=" * 80)
        self.log(f"🔄 开始第 {self.run_counter} 次运行")
        self.log(f"📁 输出文件夹: {output_folder}")
        self.log(f"🕐 开始时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        self.log(f"⏳ 剩余时间: {self.format_time(self.get_remaining_time())}")
        self.log("=" * 80)
        
        cmd = self.build_command(output_folder)
        
        try:
            # 切换到Delete根目录
            os.chdir(self.delete_root)
            
            self.log(f"执行命令: {' '.join(cmd)}")
            
            start_time = time.time()
            
            # 静默执行，只记录到日志文件
            with open(self.delete_root / f"run_{self.run_counter}_output.log", 'w') as log_f:
                process = subprocess.Popen(
                    cmd,
                    stdout=log_f,
                    stderr=subprocess.STDOUT,
                    text=True
                )
                
                # 等待进程结束
                return_code = process.wait()
            
            end_time = time.time()
            duration = end_time - start_time
            
            self.log("=" * 80)
            self.log(f"✅ 第 {self.run_counter} 次运行完成")
            self.log(f"⏱️  运行时长: {duration/60:.2f} 分钟")
            self.log(f"📊 返回代码: {return_code}")
            self.log(f"🕐 结束时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            
            # 检查输出文件夹
            output_path = self.delete_root / self.config['outdir'] / output_folder
            if output_path.exists():
                sdf_files = list(output_path.glob("*.sdf"))
                self.log(f"📦 生成文件数: {len(sdf_files)}")
            else:
                self.log(f"⚠️  警告: 输出文件夹未找到: {output_path}")
            
            self.log("=" * 80)
            
            self.run_counter += 1
            self.total_runs += 1
            
            return True
            
        except Exception as e:
            self.log("=" * 80)
            self.log(f"❌ 第 {self.run_counter} 次运行出错")
            self.log(f"错误信息: {e}")
            self.log("=" * 80)
            return False
    
    def run_continuous(self):
        """持续运行Delete模型（带tqdm进度条）"""
        self.log("🎬 开始持续运行循环...")
        
        print(f"\n🎬 开始持续运行，预计运行 {self.duration_hours} 小时")
        print(f"📊 运行进度将通过进度条显示")
        
        # 创建一个不确定次数的进度条（因为不知道会运行多少次）
        pbar = tqdm(desc="运行进度", unit="次", total=None, bar_format='{desc}: {n} 次 | 运行时间: {elapsed} | 剩余: {postfix}')
        
        try:
            while self.is_time_remaining():
                # 更新进度条
                remaining = self.format_time(self.get_remaining_time())
                pbar.set_postfix_str(remaining)
                
                # 执行一次运行
                success = self.run_delete_once()
                
                # 更新进度条计数
                pbar.update(1)
                pbar.set_description(f"已完成 {self.total_runs} 次运行")
                
                # 检查是否还有足够时间运行下一次
                if not self.is_time_remaining():
                    self.log("⏰ 已达到设定的运行时长，停止运行")
                    break
                
                # 等待后继续
                if WAIT_BETWEEN_RUNS > 0:
                    self.log(f"⏳ 等待 {WAIT_BETWEEN_RUNS} 秒后开始下一次运行")
                    time.sleep(WAIT_BETWEEN_RUNS)
                
        except KeyboardInterrupt:
            self.log("=" * 80)
            self.log("🛑 用户中断运行")
            self.log(f"📊 总共完成运行次数: {self.total_runs}")
            self.log("=" * 80)
            print(f"\n🛑 用户中断运行")
        
        finally:
            pbar.close()
            
            # 最终统计
            actual_duration = (datetime.datetime.now() - self.start_time).total_seconds() / 3600
            
            self.log("=" * 80)
            self.log("🏁 运行结束")
            self.log(f"📊 总运行次数: {self.total_runs}")
            self.log(f"⏱️  实际运行时长: {actual_duration:.2f} 小时")
            self.log(f"🕐 结束时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            self.log("=" * 80)
            
            print(f"\n{'='*80}")
            print(f"🏁 运行完成！")
            print(f"📊 总运行次数: {self.total_runs}")
            print(f"⏱️  实际运行时长: {actual_duration:.2f} 小时")
            print(f"📝 详细日志: {self.log_file}")
            print(f"{'='*80}")

# ==============================================================================
# 执行自动化运行
# ==============================================================================

# 创建自动运行管理器
runner = DeleteAutoRunnerV2(
    delete_root=DELETE_ROOT,
    config=DELETE_CONFIG,
    duration_hours=RUN_DURATION_HOURS,
    use_cpu=USE_CPU_VERSION,
    log_file=LOG_FILE
)

# 开始持续运行
runner.run_continuous()

# ==============================================================================
# 使用说明
# ==============================================================================
"""
📖 使用指南（V2版本 - 基于运行时长）：

1. 【配置参数】
   - DELETE_ROOT: Delete模型根目录
   - DELETE_CONFIG: 运行参数（蛋白、配体、输出路径等）
   - RUN_DURATION_HOURS: 运行时长（小时），例如 5、6、8 等
   
2. 【运行模式】
   - USE_CPU_VERSION = True: 使用 delete_cpu.py
   - USE_CPU_VERSION = False: 使用 delete.py
   
3. 【输出管理】
   - 每次运行创建唯一文件夹，避免覆盖
   - Delete模型输出被重定向到单独的log文件
   - 主日志文件记录所有运行信息
   
4. 【进度显示】
   - tqdm.notebook进度条显示运行次数
   - 实时显示已运行时间和剩余时间
   - 控制台输出简洁，详细信息在日志文件
   
5. 【日志文件】
   - 主日志: Delete_AutoRun.log（或自定义名称）
   - 每次运行日志: run_1_output.log, run_2_output.log...
   
6. 【停止运行】
   - 自动停止: 达到设定时长后自动结束
   - 手动停止: 点击Jupyter停止按钮
   
⚠️  注意事项：
   - 确保Delete环境已激活
   - 确保磁盘空间充足
   - 可通过日志文件查看详细运行信息
   - 建议先设置短时间（如0.5小时）测试
"""